In [ ]:
# Uploading all professor modules
import sys
sys.path.append('../GithubModules')
print(sys.version)

In [ ]:
import pandas as pd 
URM_df=pd.read_csv('../input/RecsysDataset/data_train.csv')
ICM_genre_df=pd.read_csv('../input/RecsysDataset/data_ICM_genre.csv')
ICM_subgenre_df=pd.read_csv('../input/RecsysDataset/data_ICM_subgenre.csv')
ICM_channel_df=pd.read_csv('../input/RecsysDataset/data_ICM_channel.csv')
ICM_event_df=pd.read_csv('../input/RecsysDataset/data_ICM_event.csv')
# Extract a list of users who will be present in the final submission
target_users = pd.read_csv('../input/RecsysDataset/data_target_users_test.csv')

In [ ]:
# Just for usability/readability + convert values from float to int 

URM_df.columns = ['userID','itemID','interaction']
ICM_channel_df.columns = ['itemID','featureID','value']
ICM_event_df.columns = ['itemID','featureID','value']
ICM_genre_df.columns = ['itemID','featureID','value']
ICM_subgenre_df.columns = ['itemID','featureID','value']
URM_df['interaction'] = URM_df['interaction'].astype(int)
ICM_channel_df['value'] = ICM_channel_df['value'].astype(int)
ICM_event_df['value'] = ICM_event_df['value'].astype(int)
ICM_genre_df['value'] = ICM_genre_df['value'].astype(int)
ICM_subgenre_df['value'] = ICM_subgenre_df['value'].astype(int)

In [ ]:
#Let's use properties of the sparse matrices
import scipy.sparse as sps
import numpy as np
URM_sparse = sps.coo_matrix((URM_df['interaction'].values,(URM_df['userID'].values,URM_df['itemID'].values)))
URM_csr = URM_sparse.tocsr()

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_user_wise
from Evaluation.Evaluator import EvaluatorHoldout

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr , train_percentage = 0.80)
#URM_train, URM_validation = split_train_in_two_percentage_user_wise(URM_sparse.tocsr(), train_percentage = 0.8, verbose = False)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [ ]:
from skopt.space import Real, Integer, Categorical
import numpy as np
hyperparameters_range_dictionary = {
    'epochs': Integer(130, 200),
    'num_factors': Integer(30, 55),
    'alpha': Real(0.6, 0.7),
    'reg': Real(0.0005, 0.0045)
}

In [ ]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
recommender_class = IALSRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [ ]:
import os

output_folder_path = '/Results/iALS/'

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 15
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [ ]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs
)

In [ ]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )